In [1]:
#!/usr/bin/env python

## MSC_Thesis 
## Jacotte Monroe 
## 19/03/24

## CHANGE DESCRIPTION BECAUSE THIS WAS COPY PASTED!!!
## Script that reads the spatial and temporal extents and retrieves the matching MODIS 250m images. 
## MODIS dataset is cloudmasked using a bitmask from MODIS 500m, then gap filled using temporal linear interpolation. 
## Each image is reprojected to EPSG 32733 (Namibia projection) and clipped to study area. 
## NDVI is calculated. 
## Script inputs: 
##       0_single_run_settings.csv
##       2_a1_step_extents.csv
## Script outputs: 
##       3_a1_modis_images (folder) > [daily_MODIS_NDVI_250m_date].tif (intermediate output) & 3_a2_modis_ndvi.tif (intermediate output)

## NOTE: Landsat images are large and may not get exported due to size limit. If that is the case, a manual download option is available. 
##       Run GEE code: https://code.earthengine.google.com/8b4653d8807cfa7ba185b45e18243e6d
##       The coordinates of the large region must be entered manually in the GEE script, then run, and export images in Task



###########
## Script set-up and loading necessary libraries 
###########

import ee
ee.Authenticate()

import geemap
import os
import pandas as pd
import math 

ee.Initialize()





# function to add timestamp band to image
# source: https://spatialthoughts.com/2021/11/08/temporal-interpolation-gee/
def addTimestamp(image): 
    # create new image where pixel value = time of original image
    timeImage = image.metadata('system:time_start').rename('timestamp')

    # mask new time image with original image to remove cloudmasked pixels
    timeImageMasked = timeImage.updateMask(image.mask().select(0))

    # return original image with time image as new band 
    return image.addBands(timeImageMasked)



# function that takes image and replaces masked pixels with linearly interpolated values from bef/aft images
def interpolateImage(image):
    image = ee.Image(image)

    # get list of before/after images from image property
    beforeImages = ee.List(image.get('before'))
    afterImages = ee.List(image.get('after'))

    # create image collection of before/after images
    # mosaic() combines images into one image accordint to their position in collection 
    #  image first has all pixels from last image in collection 
    #  gaps filled with second to last image from collection ...
    beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
    afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

    # rename time band of images 
    time_bef = beforeMosaic.select('timestamp').rename('time_bef')
    time_aft = afterMosaic.select('timestamp').rename('time_aft')
    time0 = image.metadata('system:time_start').rename('time0')

    # combine all three single band time images into one image with three time bands 
    timeImage = ee.Image.cat([time_bef, time_aft, time0])

    # compute image of interpolated surface reflectance values 
    timeRatio = timeImage.expression('(time0 - time_bef) / (time_aft - time_bef)', \
                    {'time0': timeImage.select('time0'), 
                     'time_bef': timeImage.select('time_bef'), 
                     'time_aft': timeImage.select('time_aft')})

    interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))

    # replace masked pixels in current image with pixels from interpolated mosaic
    result = image.unmask(interpolated)

    # return gap-filled image
    return result.copyProperties(image, ['system:time_start'])

# define landsat scaling factor
def applyScaleFactors(image): 
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)

# function to reproject (32733 if want Namibia coordinates, but elephant fixes are in 4326)
def reprojectLandsat(image): 
    return image.reproject('EPSG:32733', None, 30)

def reprojectModis(image):
    return image.reproject('EPSG:32733', None, 250)

# function to clip image to study area
# have to include nested function because can't use map() with more than one argument 
# source: https://gis.stackexchange.com/questions/473500/mapping-function-with-multiple-arguments-in-google-earth-engine
def clipToAOI(bbox): 
    def withBbox(image):
        return image.clip(bbox).copyProperties(image, ['system:id'])
    return withBbox

# function to calculate NDVI 
def addNDVI(image): 
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return image.addBands(ndvi)



def retrieveImagesForDownscaling(): 
    ###########
    ## Read run settings 
    ###########
    run_settings_table = pd.read_csv('data/single_run_settings.csv', index_col = 0, header = None)

    # define settings
    ID = run_settings_table.iloc[0,0]
    week = str(run_settings_table.iloc[0,1])
    random_data_method = run_settings_table.iloc[0,2]

    # check that the week setting doesn't preceed Landsat 8 data --> week >= 2259
    # Landsat starts 2013-03-18 meaning that week cannot be before 2013-04-18 (because 1 month buffer, see code below)
    # source: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2
    # if the week is before 2013-04-18, skip the iteration and go to next run 
    # source: https://www.tutorialspoint.com/python/python_loop_control.htm
    # CODE CURRENTLY GETTING ERROR THAT ITS NOT IN LOOP WHICH IS TRUE SO FOR NOW COMMENTED OUT AND HAVE TO MAKE SURE I GET THE RIGHT WEEKS
    #if week < 2259: 
     #   continue
    
    
    
    ###########
    ## Load step extents LUT
    ###########
    extents_lut = pd.read_csv('data/' + ID + '/' + week + '/' + '2_a1_step_extents_LUT_' + random_data_method + '.csv')
    
    
    
    ###########
    ## Create region (largest extent) geometry and define first and last dates
    ###########
    large_extent = extents_lut.iloc[-1]

    xmin = large_extent.loc['xmin']
    xmax = large_extent.loc['xmax']
    ymin = large_extent.loc['ymin']
    ymax = large_extent.loc['ymax']
    
    large_extent_coords = [[[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]]]
    
    large_region = ee.Geometry.Polygon(large_extent_coords, proj = 'EPSG:32733', evenOdd = False)
    
    first_date = ee.Date(large_extent.loc['start_date_prev_week'], 'Africa/Maputo')
    week_start_date = ee.Date(large_extent.loc['start_date'], 'Africa/Maputo')
    last_date = ee.Date(large_extent.loc['end_date'], 'Africa/Maputo')
    
    # define landsat buffer (days) to make sure there are enough Landsat images being included
    l8_buffer = ee.Number(9)

    # create smaller geometries from large extent for Landsat download (loophole around file size limitation)
    xq1 = xmin + (xmax - xmin)*0.25
    xmid = xmin + (xmax - xmin)*0.5
    xq3 = xmin + (xmax - xmin)*0.75
    yq1 = ymin + (ymax - ymin)*0.25
    ymid = ymin + (ymax - ymin)*0.5
    yq3 = ymin + (ymax - ymin)*0.75

    large_extent_coords_TL_TL = [[[xmin, yq3], [xmin, ymax], [xq1, ymax], [xq1, yq3]]]
    large_extent_coords_TL_TR = [[[xq1, yq3], [xq1, ymax], [xmid, ymax], [xmid, yq3]]]
    large_extent_coords_TL_BR = [[[xq1, ymid], [xq1, yq3], [xmid, yq3], [xmid, ymid]]]
    large_extent_coords_TL_BL = [[[xmin, ymid], [xmin, yq3], [xq1, yq3], [xq1, ymid]]]
    
    large_extent_coords_TR_TL = [[[xmid, yq3], [xmid, ymax], [xq3, ymax], [xq3, yq3]]]
    large_extent_coords_TR_TR = [[[xq3, yq3], [xq3, ymax], [xmax, ymax], [xmax, yq3]]]
    large_extent_coords_TR_BR = [[[xq3, ymid], [xq3, yq3], [xmax, yq3], [xmax, ymid]]]
    large_extent_coords_TR_BL = [[[xmid, ymid], [xmid, yq3], [xq3, yq3], [xq3, ymid]]]
    
    large_extent_coords_BR_TL = [[[xmid, yq1], [xmid, ymid], [xq3, ymid], [xq3, yq1]]]
    large_extent_coords_BR_TR = [[[xq3, yq1], [xq3, ymid], [xmax, ymid], [xmax, yq1]]]
    large_extent_coords_BR_BR = [[[xq3, ymin], [xq3, yq1], [xmax, yq1], [xmax, ymin]]]
    large_extent_coords_BR_BL = [[[xmid, ymin], [xmid, yq1], [xq3, yq1], [xq3, ymin]]]

    large_extent_coords_BL_TL = [[[xmin, yq1], [xmin, ymid], [xq1, ymid], [xq1, yq1]]]
    large_extent_coords_BL_TR = [[[xq1, yq1], [xq1, ymid], [xmid, ymid], [xmid, yq1]]]
    large_extent_coords_BL_BR = [[[xq1, ymin], [xq1, yq1], [xmid, yq1], [xmid, ymin]]]
    large_extent_coords_BL_BL = [[[xmin, ymin], [xmin, yq1], [xq1, yq1], [xq1, ymin]]]

    large_region_TL_TL = ee.Geometry.Polygon(large_extent_coords_TL_TL, proj = 'EPSG:32733', evenOdd = False)
    large_region_TL_TR = ee.Geometry.Polygon(large_extent_coords_TL_TR, proj = 'EPSG:32733', evenOdd = False)
    large_region_TL_BR = ee.Geometry.Polygon(large_extent_coords_TL_BR, proj = 'EPSG:32733', evenOdd = False)
    large_region_TL_BL = ee.Geometry.Polygon(large_extent_coords_TL_BL, proj = 'EPSG:32733', evenOdd = False)
    
    large_region_TR_TL = ee.Geometry.Polygon(large_extent_coords_TR_TL, proj = 'EPSG:32733', evenOdd = False)
    large_region_TR_TR = ee.Geometry.Polygon(large_extent_coords_TR_TR, proj = 'EPSG:32733', evenOdd = False)
    large_region_TR_BR = ee.Geometry.Polygon(large_extent_coords_TR_BR, proj = 'EPSG:32733', evenOdd = False)
    large_region_TR_BL = ee.Geometry.Polygon(large_extent_coords_TR_BL, proj = 'EPSG:32733', evenOdd = False)
    
    large_region_BR_TL = ee.Geometry.Polygon(large_extent_coords_BR_TL, proj = 'EPSG:32733', evenOdd = False)
    large_region_BR_TR = ee.Geometry.Polygon(large_extent_coords_BR_TR, proj = 'EPSG:32733', evenOdd = False)
    large_region_BR_BR = ee.Geometry.Polygon(large_extent_coords_BR_BR, proj = 'EPSG:32733', evenOdd = False)
    large_region_BR_BL = ee.Geometry.Polygon(large_extent_coords_BR_BL, proj = 'EPSG:32733', evenOdd = False)
    
    large_region_BL_TL = ee.Geometry.Polygon(large_extent_coords_BL_TL, proj = 'EPSG:32733', evenOdd = False)
    large_region_BL_TR = ee.Geometry.Polygon(large_extent_coords_BL_TR, proj = 'EPSG:32733', evenOdd = False)
    large_region_BL_BR = ee.Geometry.Polygon(large_extent_coords_BL_BR, proj = 'EPSG:32733', evenOdd = False)
    large_region_BL_BL = ee.Geometry.Polygon(large_extent_coords_BL_BL, proj = 'EPSG:32733', evenOdd = False)
    
    
    
    
    ###########
    ## Load feature of Etosha National Park (Namibia) and transform into geometry
    ###########
    # load Etosha National Park study area
    enp = ee.Feature(ee.FeatureCollection('WCMC/WDPA/current/polygons') \
            .filter(ee.Filter.eq('ORIG_NAME', 'Etosha')) \
            .first())
    
    # create a buffer around the park to retrieve images that are slightly outside of ENP
    # source: https://developers.google.com/earth-engine/apidocs/ee-feature-buffer
    # note: buffered 10km around the park
    enp = enp.buffer(10000)
    
    # turn ENP study area into a geometry
    enp_geom = enp.geometry()
    


    ###########
    ## Set interpolation parameters 
    ###########
    # set time-window for interpolation (how far will interpolate)
    # note: this function only used for downscaling, data has already been handpicked for downscaling purposely to avoid cloudcover
    #       so there aren't many consecutive days without data --> time window can be kept relatively small 
    # source: https://spatialthoughts.com/2021/11/08/temporal-interpolation-gee/
    days = ee.Number(3) 
    
    # convert to milliseconds (for gap filling step)
    millis = days.multiply(1000*60*60*24)

    
    
    ###########
    ## Define path of output directory 
    ###########
    # define output folder paths for current MODIS images and images from week prior
    out_dir_m = 'data/' + ID + '/' + week + '/' + '3_b1_modis_images_downscaling_' + random_data_method + '/'
    out_dir_l = 'data/' + ID + '/' + week + '/' + '3_b2_landsat_images_downscaling_' + random_data_method + '/'
    
    
    
    ###########
    ## Retrieve MODIS and Landsat datasets
    ###########
    # load daily MODIS 250m dataset 
    # source: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09GQ
    modis = ee.ImageCollection('MODIS/061/MOD09GQ') \
            .filterDate(first_date.advance(days.multiply(-1), 'day'), last_date.advance(days, 'day')) \
            .filterBounds(enp_geom) \
            .select(['sur_refl_b01', 'sur_refl_b02'], ['red', 'nir'])
    
    # load Landsat 8 level 2 
    # source: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2
    # source: https://developers.google.com/earth-engine/apidocs/ee-date-advance#colab-python
    # note: can't filter bounds with large_region until have reprojected image collections (meantime use enp_geom)
    # note: have to manually decide it take WRS_PATH 179 or 180 depending on where the large region is 
    # note: row 74 is necessary for LA14 datasets (LA26 doesn't need row 74)
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filterDate(first_date.advance(l8_buffer.multiply(-1), 'day'), last_date.advance(l8_buffer, 'day')) \
            .filterBounds(enp_geom) \
            .filter(ee.Filter.inList('WRS_PATH', ee.List([179]))) \
            .filter(ee.Filter.inList('WRS_ROW', ee.List([73, 74]))) 
    
    
    
    ###########
    ## Image gap filling - Linear interpolation of missing pixels 
    ###########
    # interpolation of MODIS values necessary because the MODIS image is missing data on days of Landsat coverage (fill with bef/aft days)
    # add timestamp band to each image 
    modis_withTime = modis.map(addTimestamp)
    
    # following tutorial: https://spatialthoughts.com/2021/11/08/temporal-interpolation-gee/
    # define filter to only retrieve images within the specified time-window
    maxDiffFilter = ee.Filter.maxDifference(difference = millis, 
                                            leftField = 'system:time_start', 
                                            rightField = 'system:time_start')
    
    # define filters that compare given image timestamp against other image timestamps
    # NOTE: leftField compared against rightField --> so in first filter = leftField smaller than rightField
    greaterEqFilter = ee.Filter.greaterThanOrEquals(leftField = 'system:time_start', 
                                                    rightField = 'system:time_start')
    
    lessEqFilter = ee.Filter.lessThanOrEquals(leftField = 'system:time_start', 
                                              rightField = 'system:time_start')
    
    # combined filters --> find all images before/after image that are within time-window
    filter_before = ee.Filter.And(maxDiffFilter, greaterEqFilter)
    filter_after = ee.Filter.And(maxDiffFilter, lessEqFilter)
    
    # set join parameters
    # pairs each image to group of matching elements from the second collection 
    # saves the matching elements as new property with matchesKey name, ordered by date
    # order set to have the closest image as last element in list
    # source: https://spatialthoughts.com/2021/11/08/temporal-interpolation-gee/
    # source: https://developers.google.com/earth-engine/apidocs/ee-join-saveall
    join_before = ee.Join.saveAll(
        matchesKey = 'before', 
        ordering = 'system:time_start', 
        ascending = True)
    
    join_after = ee.Join.saveAll(
        matchesKey = 'after', 
        ordering = 'system:time_start', 
        ascending = False)
    
    # join the cloudfree MODIS 250m collection with itself to get all previous/post images within time-window
    # results in image collection where each image has a property that lists all images preceeding/following image within time-window
    # source: https://spatialthoughts.com/2021/11/08/temporal-interpolation-gee/
    modis_joined_temp = join_before.apply(
        primary = modis_withTime, 
        secondary = modis_withTime, 
        condition = filter_before)
    
    modis_joined = join_after.apply(
        primary = modis_joined_temp, 
        secondary = modis_joined_temp, 
        condition = filter_after)
    
    # interpolate all images from MODIS 250m image collection
    modis_interpolated = ee.ImageCollection(modis_joined.map(interpolateImage))


    
    ###########
    ## Preprocess images
    ###########
    # apply scaling factor on surface reflectance bands of Landsat to get reflectance values instead of digital numbers
    # source: https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products
    l8_scaled = l8.map(applyScaleFactors).select(['SR_B.'])
    
    # reproject MODIS and Landsat datasets to Namibia projection 
    modis_interpolated = modis_interpolated.filterDate(first_date, last_date).select(['red', 'nir'])
    modis_reproj = modis_interpolated.map(reprojectModis)
    l8_reproj = l8_scaled.map(reprojectLandsat)
    
    # clip image
    modis_clipped = modis_reproj.map(clipToAOI(large_region))
    l8_clipped_TL_TL = l8_reproj.map(clipToAOI(large_region_TL_TL))
    l8_clipped_TL_TR = l8_reproj.map(clipToAOI(large_region_TL_TR))
    l8_clipped_TL_BR = l8_reproj.map(clipToAOI(large_region_TL_BR))
    l8_clipped_TL_BL = l8_reproj.map(clipToAOI(large_region_TL_BL))
    
    l8_clipped_TR_TL = l8_reproj.map(clipToAOI(large_region_TR_TL))
    l8_clipped_TR_TR = l8_reproj.map(clipToAOI(large_region_TR_TR))
    l8_clipped_TR_BR = l8_reproj.map(clipToAOI(large_region_TR_BR))
    l8_clipped_TR_BL = l8_reproj.map(clipToAOI(large_region_TR_BL))
    
    l8_clipped_BR_TL = l8_reproj.map(clipToAOI(large_region_BR_TL))
    l8_clipped_BR_TR = l8_reproj.map(clipToAOI(large_region_BR_TR))
    l8_clipped_BR_BR = l8_reproj.map(clipToAOI(large_region_BR_BR))
    l8_clipped_BR_BL = l8_reproj.map(clipToAOI(large_region_BR_BL))
    
    l8_clipped_BL_TL = l8_reproj.map(clipToAOI(large_region_BL_TL))
    l8_clipped_BL_TR = l8_reproj.map(clipToAOI(large_region_BL_TR))
    l8_clipped_BL_BR = l8_reproj.map(clipToAOI(large_region_BL_BR))
    l8_clipped_BL_BL = l8_reproj.map(clipToAOI(large_region_BL_BL))
    
    # add NDVI as a band to the MODIS dataset
    modis_ndvi = modis_clipped.map(addNDVI).select(['NDVI'])
    
    
    
    ###########
    ## Export MODIS and Landsat images 
    ###########
    # export MODIS image collection to local repository 
    # source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
    #geemap.ee_export_image_collection(modis_ndvi, out_dir = out_dir_m, region = large_region)
    
    # export Landsat 8 image collection to local repository 
    #geemap.ee_export_image_collection(l8_clipped_TL_TL, out_dir = out_dir_l + 'TL_TL/', region = large_region_TL_TL)
    #geemap.ee_export_image_collection(l8_clipped_TL_TR, out_dir = out_dir_l + 'TL_TR/', region = large_region_TL_TR)
    #geemap.ee_export_image_collection(l8_clipped_TL_BR, out_dir = out_dir_l + 'TL_BR/', region = large_region_TL_BR)
    #geemap.ee_export_image_collection(l8_clipped_TL_BL, out_dir = out_dir_l + 'TL_BL/', region = large_region_TL_BL)
    
    #geemap.ee_export_image_collection(l8_clipped_TR_TL, out_dir = out_dir_l + 'TR_TL/', region = large_region_TR_TL)
    #geemap.ee_export_image_collection(l8_clipped_TR_TR, out_dir = out_dir_l + 'TR_TR/', region = large_region_TR_TR)
    #geemap.ee_export_image_collection(l8_clipped_TR_BR, out_dir = out_dir_l + 'TR_BR/', region = large_region_TR_BR)
    #geemap.ee_export_image_collection(l8_clipped_TR_BL, out_dir = out_dir_l + 'TR_BL/', region = large_region_TR_BL)
    
    #geemap.ee_export_image_collection(l8_clipped_BR_TL, out_dir = out_dir_l + 'BR_TL/', region = large_region_BR_TL)
    #geemap.ee_export_image_collection(l8_clipped_BR_TR, out_dir = out_dir_l + 'BR_TR/', region = large_region_BR_TR)
    #geemap.ee_export_image_collection(l8_clipped_BR_BR, out_dir = out_dir_l + 'BR_BR/', region = large_region_BR_BR)
    #geemap.ee_export_image_collection(l8_clipped_BR_BL, out_dir = out_dir_l + 'BR_BL/', region = large_region_BR_BL)
    
    #geemap.ee_export_image_collection(l8_clipped_BL_TL, out_dir = out_dir_l + 'BL_TL/', region = large_region_BL_TL)
    #geemap.ee_export_image_collection(l8_clipped_BL_TR, out_dir = out_dir_l + 'BL_TR/', region = large_region_BL_TR)
    #geemap.ee_export_image_collection(l8_clipped_BL_BR, out_dir = out_dir_l + 'BL_BR/', region = large_region_BL_BR)
    #geemap.ee_export_image_collection(l8_clipped_BL_BL, out_dir = out_dir_l + 'BL_BL/', region = large_region_BL_BL)
    
    
    
    ###########
    ## Create and export mean MODIS NDVI image for large extent 
    ###########
    # This image will be used as basemap for visualization (no analysis done with it)
    # filter dataset to dates of week (remove week prior to movement)
    modis_week_ndvi = modis_ndvi.filterDate(week_start_date, last_date)
    
    # create mean NDVI raster
    # source: https://developers.google.com/earth-engine/guides/ic_reducing
    modis_ndvi_mean = modis_week_ndvi.mean()

    # reproject the raster in case it lost its projection (for some reason that happened?)
    modis_ndvi_mean = modis_ndvi_mean.reproject('EPSG:32733', None, 250)
    
    # export image
    geemap.ee_export_image(modis_ndvi_mean, filename = os.path.join(out_dir_m, "mean_ndvi.tif"), region = large_region, scale = 250)
    
    



# read the table with all run settings
all_runs = pd.read_csv('data/run_settings_downscaling_temp.csv', index_col = 0)

#row = all_runs.iloc[[3]]
#row.to_csv('data/single_run_settings.csv', header = False)

# run function for MODIS image retrieval
#retrieveImagesForDownscaling()

# iteratte over each entry in the table 
for i in range(len(all_runs)):
    # save the run settings to be read in the function
    row = all_runs.iloc[[i]]
    row.to_csv('data/single_run_settings.csv', header = False)

    # run function for MODIS image retrieval
    retrieveImagesForDownscaling()

Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/LA26/2260/3_b1_modis_images_downscaling_random_path_custom_distr/mean_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/LA26/2267/3_b1_modis_images_downscaling_random_path_custom_distr/mean_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/LA26/2278/3_b1_modis_images_downscaling_random_path_custom_distr/mean_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/LA14/2267/3_b1_modis_images_downscaling_random_path_custom_distr/mean_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/LA14/2278/3_b1_modis_images_downscaling_random_path_custom_distr/mean_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/LA14/2260/3_b1_modis_images_downscaling_r